<a href="https://colab.research.google.com/github/Jisang-hwang93/NLP_Class/blob/master/08%20TextRank%20Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Keyword Extraction - TextRank**

## **1. TextRank 구현**

In [1]:
# 데이터 입력
word_ls = ["딸기", "바나나", "사과", "딸기", "파인애플"]

In [ ]:
# 노드간 연결
node = list(set(word_ls))

matrix = [[0*i]*len(node) for i in range(len(node))]
for i in range(len(word_ls)-1):
    matrix[node.index(word_ls[i])][node.index(word_ls[i+1])] = 1
    matrix[node.index(word_ls[i+1])][node.index(word_ls[i])] = 1
    
matrix, node

([[0, 0, 1, 1], [0, 0, 0, 1], [1, 0, 0, 1], [1, 1, 1, 0]],
 ['사과', '파인애플', '바나나', '딸기'])

In [ ]:
# 노드별 엣지 개수
total_sum = [0] * 4

for i in range(len(matrix)):
    for j in range(len(matrix[i])):
        total_sum[i] += matrix[i][j]

total_sum

[2, 1, 2, 3]

In [ ]:
# 노드간 엣지 가중치 행렬
main_matrix = [[0*i]*len(matrix) for i in range(len(matrix))]

for i in range(len(main_matrix)):
    for j in range(len(main_matrix[i])):
        main_matrix[i][j] = matrix[i][j] / total_sum[i]

main_matrix

[[0.0, 0.0, 0.5, 0.5],
 [0.0, 0.0, 0.0, 1.0],
 [0.5, 0.0, 0.0, 0.5],
 [0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.0]]

In [ ]:
# 스코어 계산
node_sum = [0]*len(matrix)
k = 0.85

for i in range(len(main_matrix)):
    for j in range(len(main_matrix[i])):
        node_sum[i] += main_matrix[j][i]        
    node_sum[i] = node_sum[i]*k + (1-k)

node_sum

[0.8583333333333333, 0.43333333333333335, 0.8583333333333333, 1.85]

In [ ]:
matrix = [[0*i]*4 for i in range(len(matrix))]
for i in range(len(matrix)):
    for j in range(len(matrix[i])):
        matrix[i][j] = main_matrix[i][j] * node_sum[i]
        
matrix

[[0.0, 0.0, 0.42916666666666664, 0.42916666666666664],
 [0.0, 0.0, 0.0, 0.43333333333333335],
 [0.42916666666666664, 0.0, 0.0, 0.42916666666666664],
 [0.6166666666666667, 0.6166666666666667, 0.6166666666666667, 0.0]]

In [ ]:
node_sum2 = [0] * len(matrix)
for i in range(len(matrix)):
    for j in range(len(matrix[i])):
        node_sum2[i] += matrix[j][i]
    node_sum2[i] = node_sum2[i]*k + (1-k)
    
node_sum2

[1.0389583333333334,
 0.6741666666666667,
 1.0389583333333334,
 1.2479166666666668]

### **1-1. 결과 출력**

In [ ]:
print("======= 핵심 키워드 =======")
for i in range(len(matrix)):
    print("{} : {:.4f}".format(node[i], node_sum2[i]))

======= 핵심 키워드 =======
사과 : 1.0390
파인애플 : 0.6742
바나나 : 1.0390
딸기 : 1.2479


## **2. TextRank Class화**

In [49]:
class textrank():
    def __init__(self):
        self.node = []
        self.count_matrix = []
        self.total_sum = []
        self.main_matrix = []
        self.first_node_sum = []

    # 노드간 연결
    def connect_node(self, word_ls):
        self.node = list(set(word_ls))
        self.count_matrix = [[0*i]*len(self.node) for i in range(len(self.node))]
        for i in range(len(word_ls)-1):
            self.count_matrix[self.node.index(word_ls[i])][self.node.index(word_ls[i+1])] = 1
            self.count_matrix[self.node.index(word_ls[i+1])][self.node.index(word_ls[i])] = 1
            
        return self.count_matrix, self.node

    # 노드별 엣지 개수
    def count_node(self):
        self.total_sum = [0] * len(self.node)
        for i in range(len(self.count_matrix)):
            for j in range(len(self.count_matrix[i])):
                self.total_sum[i] += self.count_matrix[i][j]

        return self.total_sum

    # 노드간 엣지 가중치 행렬
    def make_mainmatrix(self):
        self.main_matrix = [[0*i]*len(self.count_matrix) for i in range(len(self.count_matrix))]
        for i in range(len(self.main_matrix)):
            for j in range(len(self.main_matrix[i])):
                self.main_matrix[i][j] = self.count_matrix[i][j] / self.total_sum[i]

        return self.main_matrix

    # 첫 스코어 계산
    def cal_first_score(self, k):
        self.first_node_sum = [0]*len(self.count_matrix)
        for i in range(len(self.main_matrix)):
            for j in range(len(self.main_matrix[i])):
                self.first_node_sum[i] += self.main_matrix[j][i]        
            self.first_node_sum[i] = self.first_node_sum[i]*k + (1-k)

        return self.first_node_sum
    
    # 행렬 계산
    def cal_matrix(self, node_sum):
        matrix = [[0*i]*len(self.node) for i in range(len(self.node))]
        for i in range(len(matrix)):
            for j in range(len(matrix[i])):
                matrix[i][j] = self.main_matrix[i][j] * node_sum[i]
        
        return matrix

    # 스코어 계산
    def cal_score(self, k, matrix):
        node_sum = [0]*len(self.count_matrix)
        for i in range(len(matrix)):
            for j in range(len(matrix)):
                node_sum[i] += matrix[j][i]        
            node_sum[i] = node_sum[i]*k + (1-k)

        return node_sum

    # 자동 실행
    def run(self, word_ls, k, num):
        self.connect_node(word_ls)
        self.count_node()
        self.make_mainmatrix()
        score = self.cal_first_score(k)
        for i in range(num):
            matrix = self.cal_matrix(score)
            score = self.cal_score(k, matrix)
            # 결과 출력
            print("======= 핵심 키워드 =======")
            for i in range(len(matrix)):
                print("{} : {:.4f}".format(self.node[i], score[i]))

### **2-1. 결과 확인**

In [50]:
# 데이터 입력
word_ls = ["딸기", "바나나", "사과", "딸기", "파인애플", "딸기", "바나나", "사과", "옥수수", "사과"]

In [51]:
tr = textrank()

In [52]:
tr.run(word_ls, 0.85, 20)

======= 핵심 키워드 =======
딸기 : 1.3069
사과 : 1.3069
파인애플 : 0.6340
바나나 : 1.1181
옥수수 : 0.6340
======= 핵심 키워드 =======
딸기 : 1.5344
사과 : 1.5344
파인애플 : 0.5203
바나나 : 0.8906
옥수수 : 0.5203
======= 핵심 키워드 =======
딸기 : 1.4055
사과 : 1.4055
파인애플 : 0.5847
바나나 : 1.0195
옥수수 : 0.5847
======= 핵심 키워드 =======
딸기 : 1.4785
사과 : 1.4785
파인애플 : 0.5482
바나나 : 0.9465
옥수수 : 0.5482
======= 핵심 키워드 =======
딸기 : 1.4372
사과 : 1.4372
파인애플 : 0.5689
바나나 : 0.9878
옥수수 : 0.5689
======= 핵심 키워드 =======
딸기 : 1.4606
사과 : 1.4606
파인애플 : 0.5572
바나나 : 0.9644
옥수수 : 0.5572
======= 핵심 키워드 =======
딸기 : 1.4473
사과 : 1.4473
파인애플 : 0.5638
바나나 : 0.9777
옥수수 : 0.5638
======= 핵심 키워드 =======
딸기 : 1.4549
사과 : 1.4549
파인애플 : 0.5601
바나나 : 0.9701
옥수수 : 0.5601
======= 핵심 키워드 =======
딸기 : 1.4506
사과 : 1.4506
파인애플 : 0.5622
바나나 : 0.9744
옥수수 : 0.5622
======= 핵심 키워드 =======
딸기 : 1.4530
사과 : 1.4530
파인애플 : 0.5610
바나나 : 0.9720
옥수수 : 0.5610
======= 핵심 키워드 =======
딸기 : 1.4516
사과 : 1.4516
파인애플 : 0.5617
바나나 : 0.9734
옥수수 : 0.5617
======= 핵심 키워드 =======
딸기 : 1.4524
사과 : 1.4